In [1]:
import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    torch.cuda.empty_cache()
else:
    device =  -1
print(device)

0


In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device)

In [3]:
import pandas as pd
labels = open('data/classes.txt').read().splitlines()
all_df = pd.read_csv("data/belief_benchmark.csv")
all_df

,text,sentiment,label
0,Also use of chemicals and machinery on their p...,"NEGATIVE, NEGATIVE",1
1,Sources believe that twenty-five to 30 percent...,UNDETERMINED,0
2,Although these government-led initiatives are ...,UNDETERMINED,0
3,"In other projects of this kind, it is not alwa...",POSITIVE,2
4,Many African policy makers evidently believe t...,POSITIVE,2
...,...,...,...
358,Similar initiatives have and can be initiated ...,POSITIVE,2
359,Collective sales account for an average of 64 ...,UNDETERMINED,0
360,Individual sales account for slightly lower vo...,"UNDETERMINED, UNDETERMINED, UNDETERMINED",0
361,Modernization of the rice value chain The VC t...,UNDETERMINED,0


In [4]:
from datasets import Dataset

dataset = Dataset.from_pandas(all_df)
dataset

Dataset({
    features: ['text', 'sentiment', 'label'],
    num_rows: 363
})

In [5]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

labels = ["neither", "negative", "positive", "both"]
pred_i_list = []
for pred in tqdm(classifier(KeyDataset(dataset, "text"), batch_size=4, candidate_labels=labels)):
    pred_i = labels.index(pred["labels"][0])
    pred_i_list.append(pred_i)

  0%|          | 0/91 [00:00<?, ?it/s]

In [6]:
from sklearn.metrics import classification_report

y_true = all_df["label"]
y_pred = pred_i_list
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

     neither       0.20      0.01      0.02       122
    negative       0.53      0.62      0.57        65
    positive       0.73      0.35      0.48       147
        both       0.05      0.38      0.09        29

    accuracy                           0.29       363
   macro avg       0.38      0.34      0.29       363
weighted avg       0.46      0.29      0.31       363

